In [4]:
!pip install ace_tools

In [1]:
# Re-import after code state reset
from faker import Faker
import pandas as pd
import sqlite3
import uuid
import random
from datetime import datetime

# Initialize Faker
fake = Faker()

# Generate synthetic customer data
def generate_customers(n=70):
    data = []
    for _ in range(n):
        customer = {
            "id": str(uuid.uuid4()),
            "full_name": fake.name(),
            "email": fake.email(),
            "phone_number": fake.phone_number(),
            "address": fake.address().replace("\n", ", "),
            "account_number": fake.unique.bban(),
            "account_type": random.choice(["Savings", "Cheque", "Business"]),
            "balance": round(random.uniform(100.0, 50000.0), 2),
            "loan_eligible": random.choice([True, False]),
            "loan_amount": round(random.uniform(0.0, 200000.0), 2),
            "last_login": fake.date_time_between(start_date="-1y", end_date="now").strftime("%Y-%m-%d %H:%M:%S"),
        }
        data.append(customer)
    return pd.DataFrame(data)

# Generate the data
df_customers = generate_customers()

# Save to SQLite database in the Jupyter environment
conn = sqlite3.connect("local_customers.db")
df_customers.to_sql("customers", conn, index=False, if_exists="replace")
conn.commit()
conn.close()

# import ace_tools as tools; tools.display_dataframe_to_user(name="Local Synthetic Customer Data", dataframe=df_customers)


In [2]:
df_customers.head()

,id,full_name,email,phone_number,address,account_number,account_type,balance,loan_eligible,loan_amount,last_login
0,b5afabf4-4df1-49c9-afed-90bea1a7ac1b,Melissa Andrews,crawforddanielle@example.net,867.353.4688x27996,"06151 Victoria Loop, Contrerasview, MD 06564",QXLP48139025131225,Business,27243.94,True,158429.05,2025-04-01 13:28:24
1,606a5eaf-a7a0-4eff-bdce-772492c45e62,Raymond Lopez,nsanford@example.org,001-337-372-4357x57156,"9876 Cruz Light Apt. 755, Oscarview, VT 78768",QSFG85951003617626,Cheque,47982.47,False,85013.18,2024-11-02 13:45:25
2,5306071b-f337-48b2-8ff5-1d08ddb09f29,Christine Christensen,jonathan85@example.com,001-959-882-9643x95479,"943 Mann Villages Apt. 214, Garciamouth, ND 46274",QFBC11725795535963,Business,15236.01,True,74324.20,2024-12-23 19:52:06
3,73927261-a2e7-4597-9b78-e041ad69a56a,Edward Bowen,sabrina95@example.net,884-451-4838,"2079 Burton Overpass Suite 409, Vincentfort, M...",ODOU39338314118591,Business,43609.10,False,4464.72,2025-02-09 13:22:39
4,27b866d7-90d2-478e-b264-abfb598ca8bf,James Rodriguez,michaelespinoza@example.net,+1-578-223-6464x3873,"958 Andrea Forks Apt. 694, Melissastad, ID 93779",EHWD87149665764245,Savings,14746.43,True,95142.21,2025-02-21 23:51:32


# Functions to be called

In [3]:
# Step 1: Set up database connection for reuse
import sqlite3

# Connect to the local SQLite DB
db_path = "local_customers.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# # Function to check if a customer exists by ID or account number
# def get_customer_by_id_or_account(identifier):
#     query = """
#     SELECT * FROM customers WHERE id = ? OR account_number = ?
#     """
#     cursor.execute(query, (identifier, identifier))
    
#     result = cursor.fetchone()
#     if result:
#         columns = [column[0] for column in cursor.description]
#         return dict(zip(columns, result))
#     return None


# Function to update customer details
def update_customer_detail(identifier, field, new_value):
    query = f"""
    UPDATE customers SET {field} = ? WHERE id = ? OR account_number = ?
    """
    cursor.execute(query, (new_value, identifier, identifier))
    conn.commit()
    return cursor.rowcount > 0

# Confirm that everything is ready
"Database connection and utility functions are set up. Ready for GPT chatbot integration."


'Database connection and utility functions are set up. Ready for GPT chatbot integration.'

# First Agent

In [5]:
import openai
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [8]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions="You are a story teller specialising in a south african language Sepedi. Respond only in sepedi",
    input="wite me a story of a sheppered looking after his cattles",
)

print(response.output_text)

Go be le mošate wa go bapalela diphoofolo, go na le mophato wa dikgomo le dihlare. Lere wa gagwe e be e le Kgabo. Kgabo o ne a rata dikgomo tša gagwe kudu, gomme a di kgathalela ka natla.

Motsemogetse, Kgabo o ne a tsamaela dikgomo go etša go tšea metsi mo ngwageng. O ne a di fa sehlare se segolo sa ditlhare go senya mošate, mme a di fa metsi ka mehla. Le fa go ntse jalo, o ne a sitwa go fa dikgomo gae, ka lebaka la diphetho le mekgwa ya go tšwa morago.

Le fa go ntse jalo, moletlo o ne o bala go tiisa motswako wa gagwe le borai ba go tlogela go tsena moetšeng. Ke nyakišiši ya go ahlaahlanya, a fa diphoofolo tša gagwe molemo go sebele le go di fa le malwetse a fosa.

Ka lebaka la serapa se se kgolo, Kgabo o ne a makatša mo mokhoa wa gagwe wa go apola. O ne a fa dikgomo le sephiri sa gago, a fa dihlare. Dilo di be di le telele, empa Kgabo o ne a sa tshwenyehe, ka lebaka la boikokobetšo le tshegofatso ya gagwe.

Dihlogong tša kgale, tselane ye ya dikgomo e ne e le senogwa; bjang o sa le

In [13]:
# Define the system prompt for the update agent
update_prompt = """
You are a helpful and friendly banking assistant chatbot.
Your job is to help users update their personal information: full name, phone number, or address.

Always begin by asking the user to provide their ID or account number.

Once you have the ID:
- Call a function to check if the user exists.
- If the user is found, ask what they want to update.
- Then ask for the new value.
- Call the update function and confirm the update.

Keep the tone professional, polite, and clear.
"""

# Sample conversation simulation using OpenAI ChatCompletion
def ask_gpt_4o(user_message, chat_history=[]):
    
    messages = [{"role": "system", "content": update_prompt}] + chat_history
    messages.append({"role": "user", "content": user_message})

    response = client.responses.create(
        model="gpt-4o-mini",  
        instructions=update_prompt,
        input = user_message,
        temperature=0.0
    )
    return response.output_text, messages

"Chat setup complete. Ready to simulate a conversation."


'Chat setup complete. Ready to simulate a conversation.'

In [14]:
first_reply, history = ask_gpt_4o("Hi, I want to update my phone number.")
first_reply

'Sure! To get started, could you please provide your ID or account number?'

In [15]:
history

[{'role': 'system',
  'content': '\nYou are a helpful and friendly banking assistant chatbot.\nYour job is to help users update their personal information: full name, phone number, or address.\n\nAlways begin by asking the user to provide their ID or account number.\n\nOnce you have the ID:\n- Call a function to check if the user exists.\n- If the user is found, ask what they want to update.\n- Then ask for the new value.\n- Call the update function and confirm the update.\n\nKeep the tone professional, polite, and clear.\n'},
 {'role': 'user', 'content': 'Hi, I want to update my phone number.'}]

In [ ]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "name": "update_details",
    "description": "Update details on sql database for a user",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        }, 
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]


response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(response.output)